In [1]:
import maup # mggg's library for proration, see documentation here: https://github.com/mggg/maup
import pandas as pd # standard python data library
import geopandas as gp # the geo-version of pandas
import numpy as np 
import os
import fiona
from statistics import mean, median
from pandas import read_csv
gp.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw' #To load KML files

**Load Precinct-level Elections Data**

In [2]:
#This Data is from the FL department of state, and can only be downloaded county by county
all_files = os.listdir("/Users/peterhorton/Documents/Redistricting_Data_Hub/Coding/pdv/vest_fl_2018/precinctlevelelectionresults2018gen")

In [ ]:
#Check that all files have the same number of columns
for i in all_files:
    ref = "./precinctlevelelectionresults2018gen/"
    file_ref = ref+i
    file_prev = pd.read_csv(file_ref,sep="\t",engine='python',index_col=None, header=None)
    print(file_prev.shape)
    
#All the files have 19 columns, so they should be good to combine

In [3]:
#Create a dataframe with the txt files
li = []
for i in all_files:
    ref = "./precinctlevelelectionresults2018gen/"
    file_ref = ref+i
    file_prev = pd.read_csv(file_ref,sep="\t",engine='python',index_col=None, header=None)
    li.append(file_prev)
frame = pd.concat(li, axis=0, ignore_index=True)
print(frame.shape)

(743093, 19)


Link to data definitions and field codes  
https://fldoswebumbracoprod.blob.core.windows.net/media/694099/precinct-level-results-data-definition-field-codes.pdf

In [4]:
print(frame.head())

    0       1      2           3                      4  5              6   \
0  HOL  Holmes  10481  11/06/2018  2018 General Election  1  Ponce de Leon   
1  HOL  Holmes  10481  11/06/2018  2018 General Election  1  Ponce de Leon   
2  HOL  Holmes  10481  11/06/2018  2018 General Election  1  Ponce de Leon   
3  HOL  Holmes  10481  11/06/2018  2018 General Election  1  Ponce de Leon   
4  HOL  Holmes  10481  11/06/2018  2018 General Election  1  Ponce de Leon   

     7   8   9   10                     11 12      13            14   15  \
0  1152   0   0   0  United States Senator     120000    Rick Scott  REP   
1  1152   0   0   0  United States Senator     120000   Bill Nelson  DEM   
2  1152   0   0   0  United States Senator     120000  WriteinVotes        
3  1152   0   0   0  United States Senator     120000     OverVotes        
4  1152   0   0   0  United States Senator     120000    UnderVotes        

          16     17   18  
0  103093132  71039  601  
1  113049868  70482 

In [5]:
#Filter down to the relevant races
office_List = ['United States Senator','Governor','Attorney General','Chief Financial Officer','Commissioner of Agriculture','Amendment No. 1: Increased Homestead Property Tax Exemption','Amendment No. 2: Limitations on Property Tax Assessments','Amendment No. 3: Voter Control of Gambling in Florida','Amendment No. 4: Voting Restoration Amendment','Amendment No. 5: Supermajority Vote Required to Impose, Authorize, or Raise State Taxes or Fees','Amendment No. 6: Rights of Crime Victims; Judges','Amendment No. 7: First Responder and Military Member Survivor Benefits; Public Colleges and Universities','Amendment No. 9: Prohibits Offshore Oil and Gas Drilling; Prohibits Vaping in Enclosed Indoor Workplaces','Amendment No. 10: State and Local Government Structure and Operation','Amendment No. 11: Property Rights; Removal of Obsolete Provision; Criminal Statutes','Amendment No. 12: Lobbying and Abuse of Office by Public Officers','Amendment No. 13: Ends Dog Racing']
filtered_frame=frame[frame[11].isin(office_List)]

#Filter out unused columns
filtered_frame = filtered_frame.drop(columns=[1,2,3,4,8,9,10,12,13],axis=1)

#Filter out Senator WriteinVotes and OverVotes and UnderVotes
no_count = ['WriteinVotes','OverVotes','UnderVotes']
filtered_frame = filtered_frame[~filtered_frame[14].isin(no_count)]

In [6]:
#Make the precinct column at least 4 digits
filtered_frame[5]=filtered_frame[5].astype(str)
filtered_frame["modified_pre"]=filtered_frame[5].str.zfill(4)

#Make a column with the 3 letter county code and the precincts
filtered_frame["Pct_std"]=filtered_frame[0]+filtered_frame["modified_pre"]

In [7]:
#Pivot the data so that each row has all the results from that precinct
pivoted_2018 = pd.pivot_table(filtered_frame, values=[18], index=["Pct_std"],columns=[11,14],aggfunc=sum)

In [8]:
#Clean up index
pivoted_2018 = pivoted_2018.reset_index()
pd.set_option('display.max_columns', None)
#display(pivoted_2018)

#Rename the columns
pivoted_2018.columns=['Pct_std','G18A10NO','G18A10YES','G18A11NO','G18A11YES','G18A12NO','G18A12YES','G18A13NO','G18A13YES','G18A01NO','G18A01YES',
                      'G18A02NO','G18A02YES', 'G18A03NO','G18A03YES','G18A04NO','G18A04YES','G18A05NO','G18A05YES',
                      'G18A06NO', 'G18A06YES','G18A07NO', 'G18A07YES','G18A09NO','G18A09YES',
                      'G18ATGRMOO', 'G18ATGOSIS','G18ATGDSHA','G18CFODRIN','G18CFORPAT', 'G18AGRRCAL','G18AGRDFRI', 
                       'G18GOVRDES','G18GOV_to_add1','G18GOV_to_add2','G18GOVDGIL','G18GOVORIC','G18GOV_to_add3','G18USSDNEL','G18USSRSCO']

#Fix governors results (they count all the no party candidates together)
pivoted_2018['G18GOVONPA']=pivoted_2018['G18GOV_to_add1']+pivoted_2018['G18GOV_to_add2']+pivoted_2018['G18GOV_to_add3']

In [9]:
#Recreate the county code
pivoted_2018["County"]=pivoted_2018["Pct_std"].str[0:3]

#Check how it looks
print(pivoted_2018.head())
pivoted_2018.shape

   Pct_std  G18A10NO  G18A10YES  G18A11NO  G18A11YES  G18A12NO  G18A12YES  \
0  ALA0001     367.0      500.0     421.0      434.0     254.0      613.0   
1  ALA0002     563.0      743.0     619.0      667.0     449.0      879.0   
2  ALA0003    1096.0     1476.0    1161.0     1348.0     730.0     1884.0   
3  ALA0004     811.0     1169.0     893.0     1068.0     618.0     1392.0   
4  ALA0005    1178.0      631.0     888.0      906.0     671.0     1181.0   

   G18A13NO  G18A13YES  G18A01NO  G18A01YES  G18A02NO  G18A02YES  G18A03NO  \
0     414.0      476.0     420.0      464.0     357.0      512.0     236.0   
1     600.0      748.0     651.0      702.0     571.0      760.0     346.0   
2    1013.0     1640.0    1141.0     1525.0     988.0     1625.0     731.0   
3     962.0     1081.0     799.0     1244.0     643.0     1382.0     635.0   
4     383.0     1554.0    1313.0      562.0    1138.0      708.0     505.0   

   G18A03YES  G18A04NO  G18A04YES  G18A05NO  G18A05YES  G18A06NO  G1

(5999, 42)

All in all there are 5,999 rows in the precinct election results dataframe. The next step is to merge these precincts with the shapefiles.

**Load Shapefiles**

The documentation mentions 4 different sources for the shapefiles:

1) Florida department of state (16 counties)  
2) 2016 VEST shapefile (17 counties)  
3) U.S. Census Bureau's 2020 Redistricting Data Program Phase 2 release (18 counties)  
4) Counties (14 counties)  

67 total counties in FL, sources for 65 listed here.  

2 remaining counties are:  Columbia, Duval  

From an email conversation w/ Brian Amos (brianamos@gmail.com) on 01/13/21, I learned that:  
    "Columbia and Duval were sent from their respective SOE offices"  
    "FL DOS shapefiles were a records request"

Resource for this: county names and three letter codes sheet   
  
['HOL' 'LEV' 'STL' 'HAR' 'CAL' 'ALA' 'HER' 'WAK' 'MON' 'POL' 'OSC' 'JAC'
 'WAL' 'DES' 'PAS' 'DIX' 'JEF' 'MRN' 'GIL' 'TAY' 'GAD' 'SUW' 'STJ' 'SEM'
 'BAK' 'SAN' 'OKA' 'PAL' 'SAR' 'BRO' 'BRE' 'CIT' 'GUL' 'HIG' 'MAN' 'IND'
 'MAD' 'PIN' 'LEO' 'LEE' 'NAS' 'FLA' 'OKE' 'CLM' 'LAK' 'UNI' 'BRA' 'DUV'
 'LAF' 'BAY' 'FRA' 'CLA' 'ORA' 'SUM' 'LIB' 'HAM' 'GLA' 'PUT' 'CLL' 'HEN'
 'MRT' 'DAD' 'HIL' 'VOL' 'CHA' 'ESC' 'WAS']
['Holmes' 'Levy' 'St. Lucie' 'Hardee' 'Calhoun' 'Alachua' 'Hernando'
 'Wakulla' 'Monroe' 'Polk' 'Osceola' 'Jackson' 'Walton' 'Desoto' 'Pasco'
 'Dixie' 'Jefferson' 'Marion' 'Gilchrist' 'Taylor' 'Gadsden' 'Suwannee'
 'St. Johns' 'Seminole' 'Baker' 'Santa Rosa' 'Okaloosa' 'Palm Beach'
 'Sarasota' 'Broward' 'Brevard' 'Citrus' 'Gulf' 'Highlands' 'Manatee'
 'Indian River' 'Madison' 'Pinellas' 'Leon' 'Lee' 'Nassau' 'Flagler'
 'Okeechobee' 'Columbia' 'Lake' 'Union' 'Bradford' 'Duval' 'Lafayette'
 'Bay' 'Franklin' 'Clay' 'Orange' 'Sumter' 'Liberty' 'Hamilton' 'Glades'
 'Putnam' 'Collier' 'Hendry' 'Martin' 'Miami-Dade' 'Hillsborough'
 'Volusia' 'Charlotte' 'Escambia' 'Washington']

**1. Florida Department of State (16 counties)**

Quote from the README

> "Alachua, Baker, Bay, Bradford, Brevard, Calhoun, Citrus, Clay, Dixie, Escambia, Hardee, Hendry, Hernando, Indian River, Lafayette and Sarasota come from the Department of State."

Note: As mentioned above, these were received by VEST via a records request. Peter called an employee (850-688-2433) of the DOS a few times, the last time being January 15th, 2021 to ask about a precinct shapefile. The FL DOS is sending a CD over with all of its precinct related archive data. At the time of writing, we have not yet recieved the files from the FL DOS.  



**2. VEST '16 (17 counties)**

Quote from the README:

> "Broward, Desoto, Gadsden, Gilchrist, Gulf, Manatee, Marion, Martin, Monroe, Nassau, Pinellas, Polk, Putnam, Santa Rosa, St. Johns, St. Lucie, and Union are unchanged from the 2016 VEST shapefile."


In [12]:
vest_16 = gp.read_file("/Users/peterhorton/Documents/Redistricting_Data_Hub/Coding/pdv/vest_fl_2018/fl_2016/fl_2016.shp")

In [13]:
#Take a look at the file
print(vest_16.head())

#Look at the unique county
print(vest_16["county"].unique())

#Make a list of relevant counties to pull
vest_2016_counties = ['BRO','DES','GAD','GIL','GUL','MAN','MRN','MRT','MON','NAS','PIN','POL','PUT',
                     'SAN','STJ','STL','UNI']

#Filter down to the relevant counties
shapefiles_vest_16=vest_16[vest_16['county'].isin(vest_2016_counties)]

#Pull the relevant columns
shapefiles_vest_16=shapefiles_vest_16[['pct','county','countypct','geometry']]

#Modify the pct column so that it contains at least 4 characters
shapefiles_vest_16["pct"]=shapefiles_vest_16["pct"].astype(str).str.zfill(4)

#Create a new unique identifier column
shapefiles_vest_16["Pct_std"]=shapefiles_vest_16["county"]+shapefiles_vest_16["pct"]

#Take a look at the new, modified file
print(shapefiles_vest_16.head())

   pct county countypct  G16PRERTru  G16PREDCli  G16PRELJon  G16PRECCas  \
0  001    DAD    DAD001         277         195           4           0   
1  010    DAD    DAD010          20          16           0           0   
2  100    DAD    DAD100         641        2893          66          17   
3  101    DAD    DAD101         679        1096          34           0   
4  102    DAD    DAD102        1100        1596          29           1   

   G16PREGSte  G16PREIDeL  G16PREoth  G16USSRRub  G16USSDMur  G16USSLSta  \
0           2           0          5         337         133           3   
1           0           0          0          30           6           0   
2          41          16         24         897        2434          50   
3           7           0         18         845         947          14   
4           4           2         22        1399        1274          16   

   G16USSOth                                           geometry  
0          5  POLYGON Z ((

In [14]:
#Filter down the 2018 election results to the relevant counties where shapefiles are from VEST '16
#(These are the only ones with a chance of matching)
elections_vest16_counties = pivoted_2018[pivoted_2018['County'].isin(vest_2016_counties)]

#See what it looks like
print(elections_vest16_counties.head())

     Pct_std  G18A10NO  G18A10YES  G18A11NO  G18A11YES  G18A12NO  G18A12YES  \
304  BROA001    1204.0     1366.0     922.0     1607.0     341.0     2262.0   
305  BROA002     288.0      422.0     248.0      451.0     120.0      585.0   
306  BROA003     341.0      649.0     326.0      641.0     167.0      824.0   
307  BROA004     300.0      464.0     290.0      473.0     168.0      618.0   
308  BROA005     342.0      570.0     321.0      583.0     114.0      824.0   

     G18A13NO  G18A13YES  G18A01NO  G18A01YES  G18A02NO  G18A02YES  G18A03NO  \
304     787.0     1855.0     837.0     1755.0     564.0     1989.0     887.0   
305     174.0      550.0     277.0      417.0     189.0      506.0     219.0   
306     222.0      785.0     439.0      531.0     279.0      674.0     263.0   
307     184.0      612.0     352.0      423.0     255.0      503.0     236.0   
308     239.0      725.0     299.0      622.0     228.0      679.0     259.0   

     G18A03YES  G18A04NO  G18A04YES  G18A05N

In [15]:
#Merge shapefile with the election results
merged_data_vest16 = pd.merge(elections_vest16_counties,shapefiles_vest_16,on=['Pct_std'],how='outer',indicator=True)
vest_16_elections_only = merged_data_vest16[merged_data_vest16["_merge"]=="left_only"]['Pct_std']
vest_16_shapefile_only = merged_data_vest16[merged_data_vest16["_merge"]=="right_only"]['Pct_std']
vest_16_both = merged_data_vest16[merged_data_vest16["_merge"]=="both"]['Pct_std']

print("There are " + str(vest_16_elections_only.count()) + " precincts that only appear in the vest_18")
print("There are " + str(vest_16_shapefile_only.count()) + " precincts that only appear in the vest_16")
print("There are " + str(vest_16_both.count()) + " precincts that were matched between the two files")

There are 11 precincts that only appear in the vest_18
There are 15 precincts that only appear in the vest_16
There are 1561 precincts that were matched between the two files


**(SKIP FOR NOW) Look at the shapefile**

In [ ]:
vest_fl_filtered=vest_16[vest_16['county'].isin(vest_2016_counties)]
filtered_2016_geom["geometry"]=filtered_2016_geom.buffer(0)
vest_fl_filtered["geometry"]=vest_fl_filtered.buffer(0)
proj = vest_fl_filtered.crs
filtered_2016_geom = filtered_2016_geom.to_crs(proj)

In [ ]:
#print(filtered_2016_geom.head())
filtered_2016_geom.plot()
print(filtered_2016_geom["county"].unique())

In [ ]:
vest_fl_filtered.plot()
print(vest_fl_filtered["county"].unique())

In [ ]:
diff = vest_fl_filtered.geom_almost_equals(filtered_2016_geom,decimal=6)

In [ ]:
diff.head()

In [ ]:
print(diff.value_counts())

**3. Census Redistricting Data Program (18 counties)**

Quote from the README:

> "Charlotte, Franklin, Glades, Hamilton, Holmes, Jackson, Jefferson, Levy, Liberty, Madison, Okeechobee, Orange, Seminole, Suwannee, Taylor, Wakulla, Walton, and Washinton come from the U.S. Census Bureau's 2020 Redistricting Data Program Phase 2 release."

In [141]:
#When downloading from the Census redistricing data program, these use a FIPS code to identify counties

fips_codes = ["12015","12037","12043","12047","12059","12063","12065","12075","12077","12079","12093",
        "12095","12117","12121","12123","12129","12131","12133"]

#Combine all the data from separate files into one
li = []
for i in fips_codes:
    ref = "./census/state_files/partnership_shapefiles_19v2_"
    file_ref = ref+i+"/PVS_19_v2_vtd_"+i+".shp"
    file_prev = gp.read_file(file_ref)
    li.append(file_prev)
shapefiles_census = pd.concat(li, axis=0, ignore_index=True)
#print(shapefiles_census.shape)

In [18]:
#Look at the data
print(shapefiles_census.head())

  STATEFP COUNTYFP   VTDST            NAMELSAD VTDI LSAD CHNG_TYPE ORIG_NAME  \
0      12      015  000001   1-Voting District    A   00      None      None   
1      12      015  000002   2-Voting District    A   00      None      None   
2      12      015  000003   3-Voting District    A   00      None      None   
3      12      015  000004   4-Voting District    A   00      None      None   
4      12      015  000016  13-Voting District    A   00      None      None   

  ORIG_CODE RELATE                NAME VINTAGE FUNCSTAT JUSTIFY  MTFCC  \
0      None   None   1-Voting District      90        N    None  G5240   
1      None   None   2-Voting District      90        N    None  G5240   
2      None   None   3-Voting District      90        N    None  G5240   
3      None   None   4-Voting District      90        N    None  G5240   
4      None   None  13-Voting District      90        N    None  G5240   

                                            geometry  
0  POLYGON Z ((-82.

In [19]:
#Make a dictionary to convert from FIPS identifier to the 3 character county name
county_code = {'015':"CHA", '037':"FRA", '043':"GLA", '047':"HAM", '059':"HOL", '063':"JAC", '065':"JEF", 
               '075':"LEV", '077':"LIB", '079':"MAD", '093':"OKE", '095':"ORA",
 '117':"SEM", '121':"SUW", '123':"TAY", '129':"WAK", '131':"WAL", '133':"WAS"}

#Create a column with this 3-character county name
shapefiles_census['county_name'] = shapefiles_census['COUNTYFP'].map(county_code)

When using "VTDST" as the matcher, the outcome was:

>There are 97 precincts that only appear in the election results  
There are 190 precincts that only appear in the shapefile  
There are 483 precincts that were matched between the two files  

When using the first two digits of "NAMELSAD" as the matcher, the outcome was:

>There are 281 precincts that only appear in the election results  
There are 374 precincts that only appear in the shapefile  
There are 299 precincts that were matched between the two files  

Leads me to believe that "VTDST" is the better column to match on

In [20]:
#Create a unique identifier, looks like "VTDST" is the best column to use for this, see above

#Work on the "NAMELSAD" column to just store this as well
shapefiles_census["NAMELSAD"]= shapefiles_census["NAMELSAD"].str.split("-", n = 1, expand = True)

#Take off the leading zero
shapefiles_census["VTDST"] = shapefiles_census["VTDST"].str.lstrip('0')

#Make sure they are all at least four digits
shapefiles_census["VTDST"] = shapefiles_census["VTDST"].str.zfill(4)
shapefiles_census["NAMELSAD"] = shapefiles_census["NAMELSAD"].str.zfill(4)

#Create the unique identifier
shapefiles_census["Pct_std"]=shapefiles_census["county_name"]+shapefiles_census["VTDST"]
shapefiles_census["alt_Pct_std"]=shapefiles_census["county_name"]+shapefiles_census["NAMELSAD"]

#Confirm that the unique identifier really is unique
print(shapefiles_census["Pct_std"].value_counts())

ORA440D    1
ORA0507    1
OKEP-01    1
HAM0007    1
ORA0413    1
          ..
HOL0004    1
WAS0004    1
ORA440A    1
ORA342B    1
CHA0057    1
Name: Pct_std, Length: 673, dtype: int64


In [21]:
#Filter down the 2018 election results to the relevant counties where shapefiles are from the Census Redist. Data Program
#(These are the only ones with a chance of matching)
census_counties = ["CHA","FRA","GLA","HAM","HOL","JAC","JEF","LEV","LIB","MAD","OKE","ORA","SEM","SUW","TAY","WAK","WAL","WAS"]
election_census=pivoted_2018[pivoted_2018["County"].isin(census_counties)]

In [22]:
#Merge the shapefile with election results
merged_data_census = pd.merge(election_census,shapefiles_census,on=['Pct_std'],how='outer',indicator=True)
census_elections_only = merged_data_census[merged_data_census["_merge"]=="left_only"]['Pct_std']
census_shapefile_only = merged_data_census[merged_data_census["_merge"]=="right_only"][['Pct_std',"alt_Pct_std"]]
census_both = merged_data_census[merged_data_census["_merge"]=="both"]['Pct_std']

print("There are " + str(census_elections_only.count()) + " precincts that only appear in the election results")
print("There are " + str(len(census_shapefile_only)) + " precincts that only appear in the shapefile")
print("There are " + str(census_both.count()) + " precincts that were matched between the two files")

There are 97 precincts that only appear in the election results
There are 190 precincts that only appear in the shapefile
There are 483 precincts that were matched between the two files


In [ ]:
#Export these to CSV to compare, it doesn't seem like there is a simple fix for these
census_elections_only_export = pd.Series(census_elections_only)
census_elections_only_export.to_csv("./census_elections_only.csv")

census_shapefile_only.to_csv("./census_shapefile_only.csv")

**4. Counties themselves (16 counties - 14 mentioned in old README and 2 others from convo)**

Quote from README:

>"Collier, Flagler, Highlands, Hillsborough, Lake, Lee, Leon, Miami-Dade, Okaloosa, Osceola, Palm Beach, Pasco, Sumter, and Volusia come from the counties."

Quote from Brian Amos on the other 2 counties:

>"Columbia and Duval were sent from their respective SOE offices"  

What I was able to find:

- Collier - (downloaded) https://www.colliervotes.com/Voting-System-Maps-Stats/Precinct-Map-Voting-Boundaries     
- Flagler - (downloaded) https://www.flaglerelections.com/For-Voters/District-Precinct-Maps   
- Highlands - (not found)  
- Hillsborough - (downloaded) https://www.votehillsborough.org/RESEARCH-DATA/Maps-Districts   
- Lake - (not found)  
- Lee - (downloaded) https://www.google.com/maps/d/viewer?mid=1FjtBs8SVp4PQjLmY09QUr9z-Pks&ll=26.559040386734967%2C-82.17803191789233&z=10  
- Leon - (downloaded) https://geodata-tlcgis.opendata.arcgis.com/datasets/election-precincts-leon-county  
- Miami-Dade  - (downloaded) https://gis-mdc.opendata.arcgis.com/datasets/precinct-1    
- Okaloosa  - (downloaded) http://www.co.okaloosa.fl.us/gis_data  
- Osceola  -  (couldn't find) https://www.voteosceola.com/en-us/Candidate-Information/Map-Files  
- Palm Beach  -  (downloaded) https://www.pbcelections.org/Records-Data/Voting-District-Maps  
- Pasco  -  (downloaded) https://www.pascocountyfl.net/342/GIS-Data-Shape-Files  
- Sumter  - (couldn't find)  
- Volusia  -  (couldn't find)  
- Columbia -  (couldn't find)   
- Duval -  (couldn't find)  


In [131]:
#For Collier, I had to load the KML into Google Earth Pro and then export it to get the precinct labels to show up
shapefiles_collier = gp.read_file("/Users/peterhorton/Documents/Redistricting_Data_Hub/Coding/pdv/vest_fl_2018/counties/Precinct Boundaries_collier.kml")
flagler_pcts = gp.read_file("/Users/peterhorton/Documents/Redistricting_Data_Hub/Coding/pdv/vest_fl_2018/counties/Flagler (2018-02-05)/VTDBLK_1_region.shp")
shapefiles_hillsborough = gp.read_file("/Users/peterhorton/Documents/Redistricting_Data_Hub/Coding/pdv/vest_fl_2018/counties/HillsCtyElections-Various Files-2017 Shapefiles-2021015-3a3/2017ShapeFiles/PRECINCT12057_region.shp")
shapefiles_lee = gp.read_file("/Users/peterhorton/Documents/Redistricting_Data_Hub/Coding/pdv/vest_fl_2018/counties/Lee County Precincts.kml",driver='KML',split="<br>")
shapefiles_leon = gp.read_file("/Users/peterhorton/Documents/Redistricting_Data_Hub/Coding/pdv/vest_fl_2018/counties/Election_Precincts_-_Leon_County-shp/Election_Precincts_-_Leon_County.shp")
shapefiles_miami = gp.read_file("/Users/peterhorton/Documents/Redistricting_Data_Hub/Coding/pdv/vest_fl_2018/counties/Miami_Precinct-shp/Precinct.shp")
shapefiles_okaloosa = gp.read_file("/Users/peterhorton/Documents/Redistricting_Data_Hub/Coding/pdv/vest_fl_2018/counties/Okaloosa County/precinct.shp")
shapefiles_palm = gp.read_file("/Users/peterhorton/Documents/Redistricting_Data_Hub/Coding/pdv/vest_fl_2018/counties/Palm Beach SOE Shapefiles 2021/Precincts 2021.shp")
shapefiles_pasco = gp.read_file("/Users/peterhorton/Documents/Redistricting_Data_Hub/Coding/pdv/vest_fl_2018/counties/Pasco VotingPrecincts_12112020_202012301214529224/VotingPrecincts_12112020.shp")

In [132]:
#Take a look at the file
#print(shapefiles_collier.head())

#The "Name" column seems like the best one to use, but needs to be cleaned
#print(shapefiles_collier["Name"])
shapefiles_collier["Name"] = shapefiles_collier["Name"].str.replace('Precinct ','')

#Make sure the name column has at least four characters
shapefiles_collier["Name"]= shapefiles_collier["Name"].str.zfill(4)

#Create the unique identifier
shapefiles_collier["Pct_std"]="CLL"+shapefiles_collier["Name"]

#Filter down the election results
election_collier = pivoted_2018[pivoted_2018["County"]=="CLL"]

#Merge data
merged_data_collier = pd.merge(election_collier,shapefiles_collier,on=['Pct_std'],how='outer',indicator=True)
collier_elections_only = merged_data_flagler[merged_data_collier["_merge"]=="left_only"]['Pct_std']
collier_shapefile_only = merged_data_flagler[merged_data_collier["_merge"]=="right_only"]['Pct_std']
collier_both = merged_data_collier[merged_data_collier["_merge"]=="both"]['Pct_std']

#Print diffrences
print("There are " + str(collier_elections_only.count()) + " precincts that only appear in the election results")
print("There are " + str(collier_shapefile_only.count()) + " precincts that only appear in the shapefile")
print("There are " + str(collier_both.count()) + " precincts that were matched between the two files")


There are 0 precincts that only appear in the election results
There are 0 precincts that only appear in the shapefile
There are 58 precincts that were matched between the two files


<ipython-input-132-c3406ce01fe7>:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  collier_elections_only = merged_data_flagler[merged_data_collier["_merge"]=="left_only"]['Pct_std']
<ipython-input-132-c3406ce01fe7>:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  collier_shapefile_only = merged_data_flagler[merged_data_collier["_merge"]=="right_only"]['Pct_std']


In [133]:
#Take a look at the shapefile
#print(flagler_pcts.head())
#print(flagler_pcts.shape)

#"PRECINCT" is the right column to use, but it isn't unique (needs to be grouped by this)
#print(flagler_pcts["PRECINCT"].value_counts())

#Group by precinct # and reset index
shapefiles_flagler = flagler_pcts.dissolve(by="PRECINCT")
shapefiles_flagler = shapefiles_flagler.reset_index()

#Edit the precinct column to contain at least 4 character
shapefiles_flagler["PRECINCT"]= shapefiles_flagler["PRECINCT"].str.zfill(4)

#Create a unique identifer
shapefiles_flagler["Pct_std"]="FLA"+shapefiles_flagler["PRECINCT"]

#Check this looks okay
#print(shapefiles_flagler.head())

#Filter down the election results
election_flagler = pivoted_2018[pivoted_2018["County"]=="FLA"]

#Merge data
merged_data_flagler = pd.merge(election_flagler,shapefiles_flagler,on=['Pct_std'],how='outer',indicator=True)
flagler_elections_only = merged_data_flagler[merged_data_flagler["_merge"]=="left_only"]['Pct_std']
flagler_shapefile_only = merged_data_flagler[merged_data_flagler["_merge"]=="right_only"]['Pct_std']
flagler_both = merged_data_flagler[merged_data_flagler["_merge"]=="both"]['Pct_std']

#Print diffrences
print("There are " + str(flagler_elections_only.count()) + " precincts that only appear in the election results")
print("There are " + str(flagler_shapefile_only.count()) + " precincts that only appear in the shapefile")
print("There are " + str(flagler_both.count()) + " precincts that were matched between the two files")

There are 1 precincts that only appear in the election results
There are 1 precincts that only appear in the shapefile
There are 25 precincts that were matched between the two files


In [134]:
#Take a look
#print(shapefiles_hillsborough.head())

#Use the "PRECINCT" column
#print(shapefiles_hillsborough["PRECINCT"].shape)

#Edit the precinct column to contain at least 4 character
shapefiles_hillsborough["PRECINCT"]= shapefiles_hillsborough["PRECINCT"].str.zfill(4)

#Create the unique identifer column
shapefiles_hillsborough["Pct_std"]="HIL"+shapefiles_hillsborough["PRECINCT"]

#Filter down the election results
election_hillsborough = pivoted_2018[pivoted_2018["County"]=="HIL"]

#Merge data
merged_data_hillsborough = pd.merge(election_hillsborough,shapefiles_hillsborough,on=['Pct_std'],how='outer',indicator=True)
hillsborough_elections_only = merged_data_hillsborough[merged_data_hillsborough["_merge"]=="left_only"]['Pct_std']
hillsborough_shapefile_only = merged_data_hillsborough[merged_data_hillsborough["_merge"]=="right_only"]['Pct_std']
hillsborough_both = merged_data_hillsborough[merged_data_hillsborough["_merge"]=="both"]['Pct_std']

#Print diffrences
print("There are " + str(hillsborough_elections_only.count()) + " precincts that only appear in the election results")
print("There are " + str(hillsborough_shapefile_only.count()) + " precincts that only appear in the shapefile")
print("There are " + str(hillsborough_both.count()) + " precincts that were matched between the two files")

There are 0 precincts that only appear in the election results
There are 0 precincts that only appear in the shapefile
There are 390 precincts that were matched between the two files


In [135]:
#Take a look
#print(shapefiles_lee.head())

#From examining the map, only concerned with those that don't start with "()", as those designate a voting place, not an entire precinct
shapefiles_lee["First_char"] = shapefiles_lee["Name"].astype(str).str[0]=="("
shapefiles_lee = shapefiles_lee[shapefiles_lee["First_char"]==False]

#Now can use the "Name" column to create the unique identifier
shapefiles_lee["Name"]= shapefiles_lee["Name"].str.zfill(4)

#Create the unique identifer column
shapefiles_lee["Pct_std"]="LEE"+shapefiles_lee["Name"]

#Filter down the election results
election_lee = pivoted_2018[pivoted_2018["County"]=="LEE"]

#Merge data
merged_data_lee = pd.merge(election_lee,shapefiles_lee,on=['Pct_std'],how='outer',indicator=True)
lee_elections_only = merged_data_lee[merged_data_lee["_merge"]=="left_only"]['Pct_std']
lee_shapefile_only = merged_data_lee[merged_data_lee["_merge"]=="right_only"]['Pct_std']
lee_both = merged_data_lee[merged_data_lee["_merge"]=="both"]['Pct_std']

#Print diffrences
print("There are " + str(lee_elections_only.count()) + " precincts that only appear in the election results")
print("There are " + str(lee_shapefile_only.count()) + " precincts that only appear in the shapefile")
print("There are " + str(lee_both.count()) + " precincts that were matched between the two files")

There are 0 precincts that only appear in the election results
There are 0 precincts that only appear in the shapefile
There are 127 precincts that were matched between the two files


In [136]:
#Take a look at the file
#print(shapefiles_leon.head())

#Edit the precinct column to contain at least 4 character
shapefiles_leon["PRECINCT"]= shapefiles_leon["PRECINCT"].str.zfill(4)

#Create the unique identifer column
shapefiles_leon["Pct_std"]="LEO"+shapefiles_leon["PRECINCT"]

#Filter down the election results
election_leon = pivoted_2018[pivoted_2018["County"]=="LEO"]

#Merge data
merged_data_leon = pd.merge(election_leon,shapefiles_leon,on=['Pct_std'],how='outer',indicator=True)
leon_elections_only = merged_data_leon[merged_data_leon["_merge"]=="left_only"]['Pct_std']
leon_shapefile_only = merged_data_leon[merged_data_leon["_merge"]=="right_only"]['Pct_std']
leon_both = merged_data_leon[merged_data_leon["_merge"]=="both"]['Pct_std']

#Print diffrences
print("There are " + str(leon_elections_only.count()) + " precincts that only appear in the election results")
print("There are " + str(leon_shapefile_only.count()) + " precincts that only appear in the shapefile")
print("There are " + str(leon_both.count()) + " precincts that were matched between the two files")

There are 0 precincts that only appear in the election results
There are 9 precincts that only appear in the shapefile
There are 155 precincts that were matched between the two files


In [137]:
#Take a look, looks like "ID" is an okay column to use
#print(shapefiles_miami.head())

#Convert the ID column to contain at least 4 character
shapefiles_miami["ID"] = shapefiles_miami["ID"].apply(str)
shapefiles_miami["ID"] = shapefiles_miami["ID"].str.zfill(4)

#Create the unique identifer column
shapefiles_miami["Pct_std"]="DAD"+shapefiles_miami["ID"]

#Filter down the election results
election_miami = pivoted_2018[pivoted_2018["County"]=="DAD"]

#Merge data
merged_data_miami = pd.merge(election_miami,shapefiles_miami,on=['Pct_std'],how='outer',indicator=True)
miami_elections_only = merged_data_miami[merged_data_miami["_merge"]=="left_only"]['Pct_std']
miami_shapefile_only = merged_data_miami[merged_data_miami["_merge"]=="right_only"]['Pct_std']
miami_both = merged_data_miami[merged_data_miami["_merge"]=="both"]['Pct_std']

#Print diffrences
print("There are " + str(miami_elections_only.count()) + " precincts that only appear in the election results")
print("There are " + str(miami_shapefile_only.count()) + " precincts that only appear in the shapefile")
print("There are " + str(miami_both.count()) + " precincts that were matched between the two files")

There are 0 precincts that only appear in the election results
There are 0 precincts that only appear in the shapefile
There are 783 precincts that were matched between the two files


In [138]:
#Take a look
#print(shapefiles_okaloosa.head())

#Looks like "NO" is the right column, convert to string and edit to contain at least 4 characters
shapefiles_okaloosa["NO"] = shapefiles_okaloosa["NO"].apply(str)
shapefiles_okaloosa["NO"] = shapefiles_okaloosa["NO"].str.zfill(4)

#Create the unique identifer column
shapefiles_okaloosa["Pct_std"]="OKA"+shapefiles_okaloosa["NO"]

#Filter down the election results
election_okaloosa = pivoted_2018[pivoted_2018["County"]=="OKA"]

#Merge data
merged_data_okaloosa = pd.merge(election_okaloosa,shapefiles_okaloosa,on=['Pct_std'],how='outer',indicator=True)
okaloosa_elections_only = merged_data_okaloosa[merged_data_okaloosa["_merge"]=="left_only"]['Pct_std']
okaloosa_shapefile_only = merged_data_okaloosa[merged_data_okaloosa["_merge"]=="right_only"]['Pct_std']
okaloosa_both = merged_data_okaloosa[merged_data_okaloosa["_merge"]=="both"]['Pct_std']

#Print diffrences
print("There are " + str(okaloosa_elections_only.count()) + " precincts that only appear in the election results")
print("There are " + str(okaloosa_shapefile_only.count()) + " precincts that only appear in the shapefile")
print("There are " + str(okaloosa_both.count()) + " precincts that were matched between the two files")

There are 0 precincts that only appear in the election results
There are 0 precincts that only appear in the shapefile
There are 52 precincts that were matched between the two files


In [139]:
#Take a look, looks like "PRECINCT" is the column to use
#print(shapefiles_palm.head())

#Edit the precinct column to contain at least 4 character
shapefiles_palm["PRECINCT"]= shapefiles_palm["PRECINCT"].str.zfill(4)

#Create the unique identifer column
shapefiles_palm["Pct_std"]="PAL"+shapefiles_palm["PRECINCT"]

#Filter down the election results
election_palm = pivoted_2018[pivoted_2018["County"]=="PAL"]

#Merge data
merged_data_palm = pd.merge(election_palm,shapefiles_palm,on=['Pct_std'],how='outer',indicator=True)
palm_elections_only = merged_data_palm[merged_data_palm["_merge"]=="left_only"]['Pct_std']
palm_shapefile_only = merged_data_palm[merged_data_palm["_merge"]=="right_only"]['Pct_std']
palm_both = merged_data_palm[merged_data_palm["_merge"]=="both"]['Pct_std']

#Print diffrences
print("There are " + str(palm_elections_only.count()) + " precincts that only appear in the election results")
print("There are " + str(palm_shapefile_only.count()) + " precincts that only appear in the shapefile")
print("There are " + str(palm_both.count()) + " precincts that were matched between the two files")

There are 5 precincts that only appear in the election results
There are 4 precincts that only appear in the shapefile
There are 868 precincts that were matched between the two files


In [140]:
#Take a look
#print(shapefiles_pasco.head())

#Seems like "Precinct" is the right column to use and that it is a unique value
#print(shapefiles_pasco["Precinct"].value_counts())

#Edit the precinct column to contain at least 4 character
shapefiles_pasco["Precinct"]= shapefiles_pasco["Precinct"].str.zfill(4)

#Create the unique identifer column
shapefiles_pasco["Pct_std"]="PAS"+shapefiles_pasco["Precinct"]

#Filter down the election results
election_pasco = pivoted_2018[pivoted_2018["County"]=="PAS"]

#Merge data
merged_data_pasco = pd.merge(election_pasco,shapefiles_pasco,on=['Pct_std'],how='outer',indicator=True)
pasco_elections_only = merged_data_pasco[merged_data_pasco["_merge"]=="left_only"]['Pct_std']
pasco_shapefile_only = merged_data_pasco[merged_data_pasco["_merge"]=="right_only"]['Pct_std']
pasco_both = merged_data_pasco[merged_data_pasco["_merge"]=="both"]['Pct_std']

#Print diffrences
print("There are " + str(pasco_elections_only.count()) + " precincts that only appear in the election results")
print("There are " + str(pasco_shapefile_only.count()) + " precincts that only appear in the shapefile")
print("There are " + str(pasco_both.count()) + " precincts that were matched between the two files")

There are 0 precincts that only appear in the election results
There are 43 precincts that only appear in the shapefile
There are 110 precincts that were matched between the two files


**Summary of Election and Precinct Data Matching**

For the election results, there are 5,999 rows.

Of these 5,999 rows:


**Load VEST File**

In [ ]:
vest_fl = gp.read_file("/Users/peterhorton/Documents/Redistricting_Data_Hub/Coding/pdv/vest_fl_2018/fl_2018/fl_2018.shp")

In [ ]:
#Look at the columns from VEST
print(vest_fl.columns)

**Validate**

In [ ]:
unavailable_from_county = #Highlands, Lake, Sumter, Osceola, Volusia, Columbia, Duval
unavailable_from_state = 

In [ ]:
#Take a look at the VEST file
print(vest_fl.head())

In [ ]:
#Check my file w/ VEST's

vtds_2018_election = pd.merge(pivoted_2018,vest_fl,on=['Pct_std'],how='outer',indicator=True)
fl_official_vtds = vtds_2018_election[vtds_2018_election["_merge"]=="left_only"]['Pct_std']
vest_vtds = vtds_2018_election[vtds_2018_election["_merge"]=="right_only"]['Pct_std']
matched_vtds = vtds_2018_election[vtds_2018_election["_merge"]=="both"]['Pct_std']

print("There are " + str(fl_official_vtds.count()) + " precincts that only appear in the downloaded file")
print("There are " + str(vest_vtds.count()) + " precincts that only appear in the vest election file")
print("There are " + str(matched_vtds.count()) + " precincts that were matched between the two files")

#In case you want to save and look at in a spreadsheet
#matched_vtds2 = pd.Series(matched_vtds)
#matched_vtds2.to_csv("./matched_precincts.csv")

#fl_official_vtds2 = pd.Series(fl_official_vtds)
#fl_official_vtds2.to_csv("./fl_18_official_precincts.csv")

#vest_vtds2 = pd.Series(vest_vtds)
#vest_vtds2.to_csv("./vest_18_precincts.csv")

In [ ]:
matched_data = vtds_2018_election[vtds_2018_election["_merge"]=="both"]
print(matched_data.head())

In [ ]:
#Check if any names don't follow the 3 county code and at least 4-digit precinct formula
vest_fl["check"]=vest_fl["County"]+vest_fl["Precnct"].astype(str).str.zfill(4)
vest_fl["confirm"]=vest_fl["Pct_std"]==vest_fl["check"]
print(vest_fl[vest_fl["confirm"]==False])

In [ ]:
#From the above, there are 5 precincts from the VEST file that don't follow the formula
election_2018_VTD_changes = {
    "UNI1A & 1B":"UNI001A",
    "UNI2A & 2B":"UNI002A",
    "UNI3A & 3B":"UNI003A",
    "UNI4A & 4C":"UNI004A",
    "UNI5A & 5C":"UNI005A",
}

pivoted_2018['Pct_std'] = pivoted_2018['Pct_std'].map(election_2018_VTD_changes).fillna(pivoted_2018['Pct_std'])

**Precincts that only appear in the VEST file**

In [ ]:
print(vest_vtds.value_counts())

#There appear to be 67 'PAL00NP' precincts and 23 others

In [ ]:
#Take a look at the "PAL00NP"
display.max_columns=True

#These all look to be 0
display(vest_fl[vest_fl['Pct_std']=="PAL00NP"])
sum(vest_fl[vest_fl['Pct_std']=="PAL00NP"]["G18GOVDGIL"])

In [ ]:
empty_precncts = ["PAL00NP","PASGULF","BRE0000","FLA0998","LEO1201","LEO1213","LEO1231","LEO1304","LEO3402","LEO4111","LEO4115","LEO5113"]
print(vtds_2018_election[(vtds_2018_election["_merge"]=="right_only")&(~vtds_2018_election["Pct_std"].isin(empty_precncts))])



In [ ]:
print(vtds_2018_election[vtds_2018_election["_merge"]=="right_only"])

In [ ]:
empty_precncts=["FLA0099","PAL8001"]

**Precincts that only appear in the FL source file**

In [ ]:
print(fl_official_vtds.value_counts())

In [ ]:
print(vtds_2018_election[vtds_2018_election["_merge"]=="left_only"])

In [ ]:
empty_precncts=["FLA0099","PAL8001"]
print(vtds_2018_election[(vtds_2018_election["_merge"]=="left_only")&(~vtds_2018_election["Pct_std"].isin(empty_precncts))])

In [ ]:
#SEM00EV may need to be parceled out
others = {"CAL0201":"CAL201/201C","MON0033":"MON0041"}

In [ ]:
display(fl_official_vtds[fl_official_vtds['Pct_std']=="MON0036"])

In [ ]:
display(vest_fl[vest_fl['Pct_std']=="MON0006"])


In [ ]:
display.max_columns=True
display(vest_fl[vest_fl['Pct_std']=="PAL00NP"])

In [ ]:
vest_fl[vest_fl['Pct_std']=="PAL00NP"].plot()

In [ ]:
print(matched_data[matched_data["Pct_std"]=="DAD0011"])

In [ ]:
#Column List should just be a list of columns to be compared with quantitative values
#Validating the various columns found in both

def validater(df,column_List):
    for i in column_List:
        left_Data = i + "_x"
        right_Data = i + "_y"
        if(sum(df[left_Data]-df[right_Data]) != 0):
            print("For " + i + " total difference is: " + str(sum(df[left_Data]-df[right_Data])))
        
column_List = ['G18A10NO','G18A10YES','G18A11NO','G18A11YES','G18A12NO','G18A12YES','G18A13NO','G18A13YES','G18A01NO','G18A01YES',
                      'G18A02NO','G18A02YES', 'G18A03NO','G18A03YES','G18A04NO','G18A04YES','G18A05NO','G18A05YES',
                      'G18A06NO', 'G18A06YES','G18A07NO', 'G18A07YES','G18A09NO','G18A09YES',
                      'G18ATGRMOO', 'G18ATGOSIS','G18ATGDSHA','G18CFODRIN','G18CFORPAT', 'G18AGRRCAL','G18AGRDFRI', 
                       'G18GOVRDES','G18GOVONPA','G18GOVDGIL','G18GOVORIC','G18USSDNEL','G18USSRSCO']


def validater_row (df, column_List):
    matching_rows = 0
    different_rows = 0
    diff_list=[]
    
    for j in range(0,len(df.index)):
        same = True
        for i in column_List:
            left_Data = i + "_x"
            right_Data = i + "_y"
            if(df.iloc[j][left_Data]-df.iloc[j][right_Data] != 0):
                #print(df.iloc[j]["Pct_std"])
                #print(left_Data)
                #print(df.iloc[j][left_Data]-df.iloc[j][right_Data])
                #print("")
                same = False
        if(same != True):
            different_rows +=1
            diff_list.append(df.iloc[j]["Pct_std"])
            
        else:
            matching_rows +=1
    print("There are ", len(df.index)," total rows")
    print(different_rows," of these rows have differences")
    print(matching_rows," of these rows are the same")
    print(diff_list)   

In [ ]:
print(vtds_2018_election[vtds_2018_election["_merge"]=="both"].head())

In [ ]:
#DAD counties appear to be off by some ratio
validater_row(matched_data,column_List)  

#MON and WAS counties appear to be off occassionally by ratios and other times by numbers

In [ ]:
diff_list = ['MON0008', 'MON0009', 'MON0010', 'MON0011', 'MON0012', 'MON0013', 'MON0014', 'MON0015', 'MON0016', 'MON0017', 'MON0018', 'MON0019', 'MON0020', 'MON0021', 'MON0022', 'MON0023', 'MON0024', 'MON0025', 'MON0028', 'MON0029', 'MON0030', 'MON0031', 'MON0032', 'WAS0011', 'WAS0012']